# IRS `inflow` and `outflow` migration at the county level: 2011-2020

#### Import Python tools

In [2]:
%load_ext lab_black

In [14]:
import pandas as pd
import geopandas as gpd
import altair as alt
import altair_grid as altgrid
import numpy as np
import us
import requests

In [4]:
alt.themes.register("grid", altgrid.theme)
alt.themes.enable("grid")

ThemeRegistry.enable('grid')

In [5]:
pd.options.display.max_columns = 1000
pd.options.display.max_rows = 1000
alt.data_transformers.disable_max_rows()

---

## Get data sources from IRS `inflow` and `outflow` files, by year

#### Inflow urls for all years

In [10]:
inflow_urls = []

for i,o in zip(range(11,20), range(12,21)):
    inflow_urls.append(f'https://www.irs.gov/pub/irs-soi/stateinflow{str(i) + str(o)}.csv')

In [11]:
inflow_urls

#### Download inflows

In [18]:
inflow_dfs = []

for inflow_url in inflow_urls:
    inflow_dfs.append(pd.read_csv(inflow_url, dtype={"fips": str}).assign(year=inflow_url))

In [29]:
pd.concat(inflow_dfs)

,y2_statefips,y1_statefips,y1_state,y1_state_name,n1,n2,AGI,year
0,1,96,AL,AL Total Migration US and Foreign,50357,103106,2167719,https://www.irs.gov/pub/irs-soi/stateinflow111...
1,1,97,AL,AL Total Migration US,49341,100777,2118535,https://www.irs.gov/pub/irs-soi/stateinflow111...
2,1,98,AL,AL Total Migration Foreign,1016,2329,49184,https://www.irs.gov/pub/irs-soi/stateinflow111...
3,1,1,AL,AL Non-migrants,1584665,3603439,87222478,https://www.irs.gov/pub/irs-soi/stateinflow111...
4,1,13,GA,GEORGIA,9726,19988,334270,https://www.irs.gov/pub/irs-soi/stateinflow111...
...,...,...,...,...,...,...,...,...
2850,56,50,VT,Vermont,30,42,2049,https://www.irs.gov/pub/irs-soi/stateinflow192...
2851,56,33,NH,New Hampshire,24,36,1432,https://www.irs.gov/pub/irs-soi/stateinflow192...
2852,56,10,DE,Delaware,18,32,859,https://www.irs.gov/pub/irs-soi/stateinflow192...
2853,56,11,DC,District Of Columbia,17,31,4000,https://www.irs.gov/pub/irs-soi/stateinflow192...


#### Outflow urls for all years

In [19]:
outflow_urls = [] 
for i,o in zip(range(11,20), range(12,21)):
    outflow_urls.append(f'https://www.irs.gov/pub/irs-soi/stateoutflow{str(i) + str(o)}.csv')

In [22]:
outflow_urls

['https://www.irs.gov/pub/irs-soi/stateoutflow1112.csv',
 'https://www.irs.gov/pub/irs-soi/stateoutflow1213.csv',
 'https://www.irs.gov/pub/irs-soi/stateoutflow1314.csv',
 'https://www.irs.gov/pub/irs-soi/stateoutflow1415.csv',
 'https://www.irs.gov/pub/irs-soi/stateoutflow1516.csv',
 'https://www.irs.gov/pub/irs-soi/stateoutflow1617.csv',
 'https://www.irs.gov/pub/irs-soi/stateoutflow1718.csv',
 'https://www.irs.gov/pub/irs-soi/stateoutflow1819.csv',
 'https://www.irs.gov/pub/irs-soi/stateoutflow1920.csv']

#### Download outflows

In [24]:
outflow_dfs = []

for outflow_url in outflow_urls:
    outflow_dfs.append(pd.read_csv(outflow_url, dtype={"fips": str}).assign(year=outflow_url))

In [28]:
pd.concat(outflow_dfs)

,y1_statefips,y2_statefips,y2_state,y2_state_name,n1,n2,AGI,year
0,1,96,AL,AL Total Migration US and Foreign,51971,107304,2109108,https://www.irs.gov/pub/irs-soi/stateoutflow11...
1,1,97,AL,AL Total Migration US,50940,105006,2059642,https://www.irs.gov/pub/irs-soi/stateoutflow11...
2,1,98,AL,AL Total Migration Foreign,1031,2298,49465,https://www.irs.gov/pub/irs-soi/stateoutflow11...
3,1,1,AL,AL Non-migrants,1584665,3603439,87222478,https://www.irs.gov/pub/irs-soi/stateoutflow11...
4,1,13,GA,GEORGIA,9920,19470,329213,https://www.irs.gov/pub/irs-soi/stateoutflow11...
...,...,...,...,...,...,...,...,...
2850,56,9,CT,Connecticut,31,54,5214,https://www.irs.gov/pub/irs-soi/stateoutflow19...
2851,56,50,VT,Vermont,16,31,937,https://www.irs.gov/pub/irs-soi/stateoutflow19...
2852,56,10,DE,Delaware,15,32,4577,https://www.irs.gov/pub/irs-soi/stateoutflow19...
2853,56,44,RI,Rhode Island,-1,-1,-1,https://www.irs.gov/pub/irs-soi/stateoutflow19...


---